# Simulation of Flexible Vehicle Routing

In [ ]:
#TODO: Why do overlapped strategies w/ and w/o algorithm yield different costs?
#TODO: Why exactly 1 trip for N=5 routing strategies

## A. Setup

In [1]:
import sys
import numpy as np
import pandas as pd
import math
import random
import time
from concorde.tsp import TSPSolver
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
print(sys.version)

3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 22:45:16) 
[Clang 9.0.1 ]


In [2]:
# GLOBAL VARIABLES
field_width = 100 # Customer location has x-coordinate in (0, field_width)
field_height = 100 # Customer location has y-coordinate in (0, field_height)
depot_x = 50 # Depot x-coordinate
depot_y = 50 # Depot y-coordinate

## B. Classes and methods

### Instance class for random demands and locations

In [3]:
class Instance():
    """A realized set of node locations and demands and the resulting routing characteristics."""
    def __init__(self, xlocs, ylocs, demands, solve_TSP=True):

        self.size = len(demands)-1
        self.demands = demands
        self.xlocs = xlocs
        self.ylocs = ylocs
        self.distances = self.calc_distance_matrix()
        self.optimal_routes = 'None'
        self.tour = 'None'
        if solve_TSP:
            self.tour = self.solve_TSP()
        
    def calc_distance_matrix(self):
        """Returns a matrix with pairwise node distances"""
        distances = np.zeros((self.size+1, self.size+1), dtype=float)
        for i in range(self.size+1):
            for j in range(self.size+1):
                new_dist = math.sqrt((self.xlocs[i]-self.xlocs[j])**2 + (self.ylocs[i]-self.ylocs[j])**2)
                distances[i,j] = new_dist
        return distances

    def update_demands(self, demands):
        self.demands = demands
    
    def update_tour(self, tour):
        self.tour = tour
        
    def get_lowerbound(self, capacity):
        """Returns a theoretical lowerbound on the optimal routing cost"""
        return (2/capacity) * sum([self.demands[i]*self.distances[0,i]
                                        for i in range(len(self.demands))])
    
    def get_fleet_size(self, route_size):
        """Returns the number of vehicles needed to visit all nodes given a fixed route size"""
        assert self.size % route_size == 0, "Number of customers must be evenly divisible by the route size."
        return int(self.size / route_size)
    
    def solve_TSP(self):
        """Defines and returns the TSP tour through all node locations"""
        solver = TSPSolver.from_data(self.xlocs, self.ylocs, 'EUC_2D')
        solution = solver.solve()
        self.tour = list(solution.tour)
        return self.tour
    
    def save_optimal_routes(self, route_list):
        self.optimal_routes = route_list

### Cost measures

In [4]:
def get_trip_count(route_list):
    """Returns number of trips in route list"""
    assert type(route_list[0]) == list, "route_list must be a list of lists (routes)"
    count = 0
    for route in route_list:
        if route != []:
            count += 1
    return count
    
def get_circular_cost(inst,segment):
    """Returns the total distance of moving from node to node within the given segment"""
    if len(segment) == 0:
        return 0
    else:
        return sum([inst.distances[segment[i],segment[i+1]] for i in range(len(segment)-1)])

def get_radial_cost(inst,segment):
    """Returns the distance Assumes vehicle travels to/from the depot at segment endpoints."""
    if len(segment) == 0:
        return 0
    else:
        return inst.distances[0,segment[0]] + inst.distances[0,segment[-1]]

def get_total_cost(inst,segment):
    """Returns sum of circular and radial costs for the given segment"""
    return get_circular_cost(inst,segment)+get_radial_cost(inst,segment)

### VRP heuristic
Using Google OR-Tools.

In [5]:
def solve_VRP(inst, capacity):

    def create_data_model(inst, capacity):
        data = {}
        data['distance_matrix'] = inst.distances
        data['demands'] = inst.demands
        data['vehicle_capacities'] = [capacity]*inst.size
        data['num_vehicles'] = inst.size
        data['depot'] = 0
        return data

    def get_routes(solution, routing, manager):
        """Get vehicle routes from a solution and store them in an array."""
        # Get vehicle routes and store them in a two dimensional array whose
        # i,j entry is the jth location visited by vehicle i along its route.
        routes = []
        for route_nbr in range(routing.vehicles()):
            index = routing.Start(route_nbr)
            route = [manager.IndexToNode(index)]
            while not routing.IsEnd(index):
                index = solution.Value(routing.NextVar(index))
                route.append(manager.IndexToNode(index))
            routes.append(route)
        return routes

    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]
    
    
    # --- RUN PROGRAM ---

    # Zero cost if no demands
    if all(dem == 0 for dem in inst.demands):
        return (0, 0, 0)
    
    # Set up data model
    data = create_data_model(inst, capacity)
    
    # Create the routing index manager
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
    
    # Create routing model
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add capacity constraint
    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem
    solution = routing.SolveWithParameters(search_parameters)
    all_routes = get_routes(solution, routing, manager)
    nonempty_routes = [route for route in all_routes if not all(i == 0 for i in route)]
    
    # Remove the depot from the optimal routes
    #parsed_routes = [route[1:-1] for route in nonempty_routes]
    #return parsed_routes
    
    return solution.ObjectiveValue(), len(nonempty_routes)  # returns (optimal cost, number of trips)

def solve_SDVRP(inst,capacity):
    """Creates equivalent demand/location instance with unit demand and solves the VRP with splittable demands"""
    split_xlocs = [[0]] + [[inst.xlocs[i]]*inst.demands[i] for i in range(1,len(inst.demands))]
    split_ylocs = [[0]] + [[inst.ylocs[i]]*inst.demands[i] for i in range(1,len(inst.demands))]
    split_demands = [[0]] + [[1]*inst.demands[i] for i in range(1,len(inst.demands))]

    split_xlocs = [v for sublist in split_xlocs for v in sublist]
    split_ylocs = [v for sublist in split_ylocs for v in sublist]
    split_demands = [v for sublist in split_demands for v in sublist]
    split_inst = Instance(split_xlocs, split_ylocs, split_demands)

    return solve_VRP(split_inst, capacity)

### Routing algorithms and helper methods

Create skeleton (a priori) routes

In [6]:
def get_primary_routes(inst, route_size):
    """Splits customer sequence into segments of 'route_size' number of customers"""
    
    tour =  inst.tour[1:] # Exclude depot
    routes = []
    for i in range(0,len(tour),route_size):
        new_route = tour[i:i+route_size]
        routes.append(new_route)
    return routes

def get_extended_routes(inst, route_size, overlap_size):
    """Splits customer sequnce into segments of 'route_size + overlap_size' number of customers, where adjacent
    segments SHARE overlap_size number of customers."""
    tour = inst.tour[1:]
    routes = []
    for i in range(0,len(tour),route_size):
        new_route = tour[i:i+route_size+overlap_size]
        routes.append(new_route)
    return routes

Get realized trips between depot visits

In [7]:
def create_full_trips(inst, route_list, capacity, route_size=None, overlap_size=None):
    """Splits a sequence of customers into individual trips. Returns a list of lists. Arguments route_size
    and overlap_size are set to large (non-constraining) values by default if not specified."""
    
    assert type(route_list[0]) == list, "route_list must be a list of lists (routes)"
    
    # Set route_size and overlap_size to large (non-constraining value) if not specified
    if not route_size:
        route_size = 2*inst.size
    if not overlap_size:
        overlap_size = 2*inst.size
           
    # dictionary for tracking demand filled at all customers with non-zero demand
    all_dict = dict([(inst.tour[i],0) for i in range(1,len(inst.tour)) if inst.demands[inst.tour[i]] !=0])
    
    segments = []
    for route in route_list:
        i = 0
        # dictionary for customers and demand filled on the current trip
        seg_dict = {}
        while i < len(route):
            cust = route[i]
            for d in range(1,inst.demands[cust]+1):
                
                # (Relevant to overlapping routes) If previous route's vehicle handled all of this route's
                # primary customers, DON'T send this route's vehicle to the extended route (which is the
                # next route's vehicle's primary route). Instead just end this route and have the next
                # vehicle start with at the beginning of its primary route.
                if sum(seg_dict.values()) == 0 and i == route_size:
                    # force to move to next route
                    i = len(route) 
                    break
                
                # Handle customer's remaining demand
                if all_dict[cust] == inst.demands[cust]:
                    # already full served
                    pass
                elif cust not in seg_dict:
                    # begin service
                    seg_dict[cust] = 1
                    all_dict[cust] += 1
                else:
                    # continue service
                    seg_dict[cust] += 1
                    all_dict[cust] += 1
                
                # End trip if vehicle at capacity
                if (sum(seg_dict.values()) == capacity):
                    seg_array = list(seg_dict)
                    segments.append(seg_array)
                    seg_dict = {}
                    if i+1 > route_size:
                        # force to move to next route
                        i = len(route)
                        break
            i+=1
        seg_array = list(seg_dict) 
        segments.append(seg_array) # include last route
    return segments

In [9]:
def implement_k_overlapped_alg(inst, primary_routes, extended_routes, capacity, route_size, overlap_size):
    """Implement's general k-overlapped routing algorithm. Returns list of realized vehicle routes. """
    assert type(primary_routes[0]) == list, "primary_routes must be a list of lists (routes)"
    assert type(extended_routes[0]) == list, "extended_routes must be a list of lists (routes)"

    # Get overlapped segments (note that last route does not have any shared customers at the route's end)
    overlapped_segments = []
    for j in range(len(primary_routes)-1):
        new_segment = [c for c in extended_routes[j] if c not in primary_routes[j]]
        overlapped_segments.append(new_segment)

    # Initialize arrays
    primary_demands = np.asarray([sum(inst.demands[cust] for cust in route) for route in primary_routes]) # a priori primary route demand for each vehicle
    extended_demands = np.asarray([sum(inst.demands[cust] for cust in route) for route in extended_routes]) # a priori extended route demand for each vehicle                            
    overlap_demands = extended_demands - primary_demands # demands of customers in k-overlapped regions for each vehicle
                                    
    first = np.asarray([route[0] for route in primary_routes]) # first customer in route for each vehicle
    last = np.asarray([route[-1] for route in overlapped_segments] + [inst.tour[-1]])

    excess = np.zeros(len(primary_routes)) # surplus capacity for each vehicle (updated below)
    workload = np.zeros(len(primary_routes)) # demand ultimately filled by each vehicle (updated below)
    
    realized_routes = []
    
    # Loop through vehicles
    for j in range(len(primary_routes)):
        
        if j == 0:
            workload[j] = primary_demands[j] 
        else:
            workload[j] = max(0, primary_demands[j] - excess[j-1])
        
        excess[j] = min(capacity * np.ceil(float(workload[j]) / capacity) - workload[j], overlap_demands[j])
        remaining_surplus = excess[j]
        
        i = 0
        while remaining_surplus > 0:
            if i < len(overlapped_segments[j]):
                # fill demand of next shared customer
                # override default first and last customer if appropriate
                remaining_surplus -= inst.demands[overlapped_segments[j][i]]
                # set first and last customers
                if remaining_surplus == 0:
                    last[j] = overlapped_segments[j][i]
                    if i != len(overlapped_segments[j]) - 1:
                        first[j+1] = overlapped_segments[j][i+1]
                    else:
                        first[j+1] = 0 # next vehicle doesn't need to leave depot
                elif remaining_surplus < 0:
                    # vehicles will split this customer
                    last[j] = overlapped_segments[j][i]
                    first[j+1] = overlapped_segments[j][i]
            i += 1

    # Determine realized routes based on updated first and last customers
    for j in range(len(primary_routes)):
        
        # Create vehicle route
        if first[j] == 0:
            route = [] # vehicle doesn't leave depot
        else:
            first_index = inst.tour.index(first[j])
            last_index = inst.tour.index(last[j])
            route = inst.tour[first_index:last_index+1]
        
        segments = create_full_trips(inst, [route], capacity)
        # Remove customers with 0 demand
        #nonzero = [segment[i] for i in range(len(segment)) if inst.demands[segment[i]] != 0]
        
        # Append to route list
        realized_routes = realized_routes + segments
    
    return realized_routes

## C. Example Instance

### 1) Set network parameters

In [13]:
num_cust = 5 # number of customers
dmin = 0 # a customer's minimum random demand (for Uniform distribution)
dmax = 8 # a customer's maximum random demand (for Uniform Distribution)
capacity = 20 # vehicle capacity
route_size = 5 # number of customers on a truck's primary route
overlap_size = 5 # number of shared customers between two neighboring routes

print('--- PARAMETERS ---')
print('Num. of customers: {}, Demand range: [{},{}], Veh. capacity: {}, Primary route size: {}, Overlap size: {}'. \
      format(num_cust, dmin, dmax, capacity, route_size, overlap_size))

--- PARAMETERS ---
Num. of customers: 5, Demand range: [0,8], Veh. capacity: 20, Primary route size: 5, Overlap size: 5


### 2) Generate random instance

In [15]:
cust_x = field_width*np.random.random(num_cust) # x coordintes of all customers
cust_y = field_height*np.random.random(num_cust) # y coordinates of all customers
cust_dems = list(np.random.randint(dmin,dmax,num_cust)) # Uniformly distributed customer demands
xlocs = list(np.append([depot_x], cust_x)) # depot and customer x-coords
ylocs = list(np.append([depot_y], cust_y)) # depot and customer y-coords
demands = list(np.append([0], cust_dems)) # depot and customer demands
inst = Instance(xlocs, ylocs, demands)

print(inst)

### 3) Print instance summary

In [78]:
inst = baseline_sim

In [79]:
print('--- INSTANCE SUMMARY ---')
print('Number of customers: \t', inst.size)
print('Vehicle capacity: \t', capacity)
print('Primary route size: \t', route_size)
print('Overlap size: \t\t', overlap_size)
print('Big TSP tour: \t\t', inst.tour[1:])
print('Tour demands: \t\t', [inst.demands[c] for c in inst.tour[1:]])
print()
print('Reoptimized Cost: \t', solve_SDVRP(inst, capacity)[0])
print('Lowerbound Cost: \t', int(inst.get_lowerbound(capacity)))

--- INSTANCE SUMMARY ---
Number of customers: 	 5
Vehicle capacity: 	 20
Primary route size: 	 5
Overlap size: 		 5
Big TSP tour: 		 [2, 1, 3, 5, 4]
Tour demands: 		 [3, 5, 0, 5, 0]

Reoptimized Cost: 	 195
Lowerbound Cost: 	 50


### 4) Solve routing problem
#### a) Dedicated routing strategy
*The algorithm is described below:*
- Identify a single tour through all *N* customers.
- Split this sequence of *N* customers into into *M* "primary" route segments with equal number of customers ("route_size" or N' customers). This assumes *N* is divisible by *M*.
- Assign a single vehicle (with capacity *Q*) to each route.
- Get information on customers' updated (realized) demands
- Upon learning demands, vehicle *m* executes its primary route *m* in the following manner.
    - The vehicle departs from depot at full capacity.
    - It sequentially visits the customers in its primary route, skipping customers that have 0 demand.
    - Upon exhausting its capacity, the vehicle detours to the depot to refill to full capacity and resumes its route wherever it left off.
    - Upon filling all customer demands in the primary route, the vehicle has completed its route and returns to the depot.


In [80]:
print('--- DEDICATED ROUTING ---')
routes = get_primary_routes(inst, route_size)
realized = create_full_trips(inst,routes,capacity)
print('\nSkeleton routes:', *routes, sep="\n")
print('\nRealized trips:', *realized, sep="\n")
print('\nTrip count:\t', get_trip_count(realized))
print('Radial cost:\t', sum([get_radial_cost(inst,seg) for seg in realized]).round(1))
print('Circular cost:\t', sum([get_circular_cost(inst,seg) for seg in realized]).round(1))
print('Total cost:\t', sum([get_total_cost(inst,seg) for seg in realized]).round(1))

--- DEDICATED ROUTING ---

Skeleton routes:
[2, 1, 3, 5, 4]

Realized trips:
[2, 1, 5]

Trip count:	 1
Radial cost:	 73.9
Circular cost:	 83.9
Total cost:	 157.8


#### b) Overlapped routing strategy

*The algorithm is described below:*
- Create primary routes as described above in dedicated routing.
- Additionally, create extended routes by assigning each vehicle to all of the customers in its primary route plus some of the customers in a neighboring vehicle's primary route.
    - Extended routes formed under "full adjacent overlapping" means vehicle $m$ is assigned to customers in primary route $m$ plus all customers in primary route $m+1$, for vehicles $m=1,2,...,M-1$.
    - Alternatively, extended routes formed under "partial adjacent overlapping" means vehicle $m$ is assigned to customers in primary route $m$ plus $k$ additional customers in primary route $m+1$, for vehicles $m=1,2,...,M-1$.
    - In both cases, vehicle $m=M$ is assigned to only its primary route.
- Upon learning customer demands, vehicle *m* executes its extended route in the following manner.
    - The vehicle departs from depot at full capacity.
    - It sequentially visits the customers in its primary route, skipping customers that have 0 demand or demand that has already been filled. The vehicle reloads at the depot as needed.
    - Upon filling demand of the final customer in the primary route, the vehicle either (a) permanently returns to the depot if no capacity remains or (b) proceeds with the customers in the extended route and fills those customer demands until the leftover vehicle capacity is exhausted, at which point the vehicle permanently returns to the depot.
    - Vehicle *m+1* then starts primary route *m+1* wherever  vehicle *m* left off.
    - However, in the case that vehicle *m-1* satisfied the demand of ALL customers in the extended route (meaning it covered all demand in vehicle *m*'s primary route), then vehicle *m* is never deployed. Instead vehicle *m+1* just starts its route at the beginning of primary route *m+1*.

Note that a central planner assesses the realized customer demands and coordinate each vehicle's starting and ending customers (realized routes) within the extended route prior to the vehicle's departing the depot.

In [81]:
print('--- OVERLAPPED ROUTING (Based on Algorithm) ---')
primary_routes = get_primary_routes(inst, route_size)
extended_routes = get_extended_routes(inst, route_size, overlap_size)
realized = implement_k_overlapped_alg(inst,primary_routes,extended_routes,capacity,route_size,overlap_size)
print('\nSkeleton routes:', *extended_routes, sep="\n")
print('\nRealized trips:', *realized, sep="\n")
print('\nTrip count:\t', get_trip_count(realized))
print('Radial cost:\t', sum([get_radial_cost(inst,seg) for seg in realized]).round(1))
print('Circular cost:\t', sum([get_circular_cost(inst,seg) for seg in realized]).round(1))
print('Total cost:\t', sum([get_total_cost(inst,seg) for seg in realized]).round(1))

--- OVERLAPPED ROUTING (Based on Algorithm) ---

Skeleton routes:
[2, 1, 3, 5, 4]

Realized trips:
[2, 1, 5]

Trip count:	 1
Radial cost:	 73.9
Circular cost:	 83.9
Total cost:	 157.8


In [82]:
print('--- OVERLAPPED ROUTING (Alt. Method) ---')
print('Note: Implementation does not follow algorithm from paper')
routes = get_extended_routes(inst, route_size, overlap_size)
realized = create_full_trips(inst,routes,capacity,route_size,overlap_size)
print('\nSkeleton routes:', *routes, sep="\n")
print('\nRealized trips:', *realized, sep="\n")
print('\nTrip count:\t', get_trip_count(realized))
print('Radial cost:\t', sum([get_radial_cost(inst,seg) for seg in realized]).round(1))
print('Circular cost:\t', sum([get_circular_cost(inst,seg) for seg in realized]).round(1))
print('Total cost:\t', sum([get_total_cost(inst,seg) for seg in realized]).round(1))

--- OVERLAPPED ROUTING (Alt. Method) ---
Note: Implementation does not follow algorithm from paper

Skeleton routes:
[2, 1, 3, 5, 4]

Realized trips:
[2, 1, 5]

Trip count:	 1
Radial cost:	 73.9
Circular cost:	 83.9
Total cost:	 157.8


#### c) Fully flexible routing strategy

*The algorithm is described below:*
- As in the dedicated and overlapped routing strategies, create a large tour through all customers.
- Select a starting customer and send out a vehicle at full capacity to sequentially fill as much customer demand as possible.
- The vehicle returns to the depot upon exhuasting its capacity, and the next vehicle continues with the customer sequence where the previous vehicle left off.


Note that as in the overlapped routing strategies, demands are learned in advance, which means each truck's starting and ending customers can be determined prior to the day's deliveries so that vehicles can execute their realized routes simultaneously.

Also note that this strategy differs from full reoptimization since the sequence of customer visits within individul trips is fixed. In reoptimization, on the other hand, an individual vehicle trip can consists of any of the *N* customers.

In [83]:
print('--- FULLY FLEXIBLE ROUTING ---')
realized = create_full_trips(inst,[inst.tour],capacity)
print('\nRealized trips:', *realized, sep="\n")
print('\nTrip count:\t', get_trip_count(realized))
print('Radial cost:\t', sum([get_radial_cost(inst,seg) for seg in realized]).round(1))
print('Circular cost:\t', sum([get_circular_cost(inst,seg) for seg in realized]).round(1))
print('Total cost:\t', sum([get_total_cost(inst,seg) for seg in realized]).round(1))

--- FULLY FLEXIBLE ROUTING ---

Realized trips:
[2, 1, 5]

Trip count:	 1
Radial cost:	 73.9
Circular cost:	 83.9
Total cost:	 157.8


## D. Simulation

### Helper methods

In [41]:
def create_instances(scenario, num_cust, cust_sims, dem_sims):
    """Returns cust_sims by dem_sims array of Instances"""
    
    np.random.seed(1)
    
    def gen_new_instance(num_cust, scenario):

        # Generate customer locations
        new_xlocs = field_width*np.random.random(num_cust) # x coordinates of all customers
        new_ylocs = field_height*np.random.random(num_cust) # y coordinates of all customers
        
        # Generate demands depending on scenario
        if scenario == 'baseline':
            new_dems = list(np.random.randint(0,8,num_cust)) # Uniformly distributed between 0 and 8
        
        # Return new instance
        new_xlocs = list(np.append([depot_x], new_xlocs)) # include depot in customer x-coords
        new_ylocs = list(np.append([depot_y], new_ylocs)) # include depot in customer y-coords
        new_dems = list(np.append([0], new_dems)) # include depot in customer demands
        return Instance(new_xlocs, new_ylocs, new_dems)
    
    def update_demands(inst, scenario):
        # Creates copy of instance with updated demands depending on scenario
        if scenario == 'baseline':
            new_dems = list(np.random.randint(0,8,num_cust)) # Uniformly distributed between 0 and 8
        new_dems = list(np.append([0], new_dems)) # include depot in customer demands
        new_inst = Instance(inst.xlocs, inst.ylocs, new_dems, solve_TSP=False)
        new_inst.tour = inst.tour
        return new_inst
    
    # Create instance array with new customer instances
    instances = [[None for j in range(dem_sims)] for i in range(cust_sims)]
    customer_instances = [gen_new_instance(num_cust, scenario) for i in range(cust_sims)]
    
    # Create demand instances for each customer instance
    for i in range(cust_sims):
        for j in range(dem_sims):
            instances[i][j] = update_demands(customer_instances[i], scenario)

    return instances

In [45]:
def set_best_tours(demand_instances, routes, capacity, route_size=None, overlap_size=None):
    """Updates the tour of all instances to the sequence that minimizes the average cost of the routes over all demand instances.
    Assumes all instances in list demand_instances have identical customer locations."""
    
    # Get any customer instance
    inst = demand_instances[0]

    # Set current tour and cumulative cost over all demand instances as best so far
    # Note: cumulative cost yields same tour ranking as average cost across demand instances
    best_tour = inst.tour
    realized_routes = create_full_trips(inst, routes, capacity, route_size, overlap_size)
    lowest_cumul_cost = sum([get_total_cost(inst,seg) for seg in realized_routes for inst in demand_instances])

    # Copy of tour (for rotating below)
    tour = inst.tour
    
    # Loop over all customers
    for c in range(inst.size):

        # Rotate tour by one customer (keeps depot at very first spot)
        tour = tour[0:1] + tour[2:-1] + tour[1:2]
        tour_cost = 0

        # Get cumulative cost over all demand instances
        for inst in demand_instances:
            realized_routes = create_full_trips(inst, routes, capacity, route_size, overlap_size)
            for segment in realized_routes:
                tour_cost += get_total_cost(inst, segment)

        if tour_cost < lowest_cumul_cost:
            # Set as new best tour and cost
            best_tour = tour
            lowest_cumul_cost = tour_cost

    # Update tour for all demand instances in this customer row
    for inst in demand_instances:
        inst.update_tour(best_tour)
    
    return

In [122]:
def simulate(scenario, problem_sizes, capacity, route_size, overlap_size, cust_sims, dem_sims):
    
    # Start timers
    start = time.time()
    pt = 0
    dt = 0
    ot = 0
    ota = 0
    ft = 0
    st = 0
    
    def create_report(num_cust, strategy, realized_routes):
        trips = [scenario, num_cust, strategy, 'trip count', get_trip_count(realized_routes)]
        radial_cost = [scenario, num_cust, strategy, 'radial cost', sum([get_radial_cost(inst,seg) for seg in realized_routes])]
        circular_cost = [scenario, num_cust, strategy, 'circular cost', sum([get_circular_cost(inst,seg) for seg in realized_routes])]
        total_cost = [scenario, num_cust, strategy, 'total cost', sum([get_total_cost(inst,seg) for seg in realized_routes])]
        return pd.DataFrame(data = [trips, radial_cost, circular_cost, total_cost],
                                    columns = ['Scenario','Number of Customers','Routing Strategy','Metric','Value'])
    
    
    # Create timestamp for backup outputs
    timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")

    # Print simulation parameters
    print('--- SIMULATION PARAMETERS ---')
    print('Scenario Name:', scenario)
    print('Problem sizes:', problem_sizes)
    print('Vehicle capacity:', capacity)
    print('Primary route size:', route_size)
    print('Overlap size:', overlap_size)
    print('Customer instances:', cust_sims)
    print('Demand instances:', dem_sims)
    print()
    
    # Initialize arrays to store results
    sim_results = pd.DataFrame(columns = ['Scenario','Number of Customers','Routing Strategy','Metric','Value'])
   
    # Loop through each problem size
    for num_cust in problem_sizes:
       
        print('Starting problems of size {}'.format(num_cust))

        new_pt = time.time()
        # Create all customer and demand instances for this problem size
        print('Creating customer instances')
        instances = create_instances(scenario, num_cust, cust_sims, dem_sims)
        
        # Find cost minimizing starting customer / tour sequence for each set of customer locations
        print('Finding best tour across demand sets')
        for row in instances:
            inst = row[0] # customer instance
            extended_routes = get_extended_routes(inst, route_size, overlap_size)
            set_best_tours(row, extended_routes, capacity, route_size, overlap_size) # set cost-minimizing sequence
        pt += time.time() - new_pt
        
        # Loop through instances and find instance cost for different strategies
        
        for i in range(cust_sims):
            print('Starting customer instance {}...'.format(i))
            for j in range(dem_sims):

                # Get instance from array
                inst = instances[i][j]
                
                try:
                    new_dt = time.time()
                    # Solve dedicated routing
                    primary_routes = get_primary_routes(inst, route_size)
                    realized_routes = create_full_trips(inst,primary_routes,capacity)
                    sim_results = sim_results.append(create_report(num_cust, 'dedicated', realized_routes), ignore_index = True)
                    dt += time.time() - new_dt
                    
                    new_ot = time.time()
                    # Solve overlapped routing (w/o algorithm)
                    extended_routes = get_extended_routes(inst, route_size, overlap_size)
                    realized_routes = create_full_trips(inst, extended_routes, capacity, route_size, overlap_size)
                    sim_results = sim_results.append(create_report(num_cust, 'overlapped (w/o alg)', realized_routes), ignore_index = True)
                    ot += time.time() - new_ot
                    
                    new_ota = time.time()
                    # Solve overlapped routing (w/ algorithm)
                    primary_routes = get_primary_routes(inst, route_size)
                    extended_routes = get_extended_routes(inst, route_size, overlap_size)
                    realized_routes = implement_k_overlapped_alg(inst, primary_routes, extended_routes, capacity, route_size, overlap_size)
                    sim_results = sim_results.append(create_report(num_cust, 'overlapped', realized_routes), ignore_index = True)
                    ota += time.time() - new_ota
                    
                    new_ft = time.time()
                    # Solve fully flexible routing
                    realized_routes = create_full_trips(inst,[inst.tour],capacity)
                    sim_results = sim_results.append(create_report(num_cust, 'fully flexible', realized_routes), ignore_index = True)
                    ft += time.time() - new_ft
                    
                    # Solve reoptimization
                    #cost, trips = solve_SDVRP(inst, capacity)
                    #new_rows = pd.DataFrame([[scenario, num_cust, 'reoptimization', 'total cost', cost],
                    #                        [scenario, num_cust, 'reoptimization', 'trip count', trips]],
                    #                        columns = ['Scenario','Number of Customers','Routing Strategy','Metric','Value'])
                    #sim_results = sim_results.append(new_rows)
                    
                    
                except Exception as e:
                    print('ERROR: {}'.format(e))
                    print('WARNING: Simulation failed to complete. Returning last Instance object.')
                    return inst
            
            new_st = time.time()
            # Save backup of data
            sim_results.to_csv('temp/backup_{}.csv'.format(timestamp))
            st += time.time() - new_st
                    
            end = time.time()
            print('Complete. Time elapsed: {:.2f} s'.format(end-start))
    
    print('Simulation complete.')
    print()
    print('--- RUNTIME BREAKDOWN ---')
    print('Setup: {:.2f} s'.format(pt))
    print('Dedicated: {:.2f} s'.format(dt))
    print('Overlapped: {:.2f} s'.format(ot))
    print('Overlapped (w/ algorithm) {:.2f} s'.format(ota))
    print('Full Flex.: {:.2f} s'.format(ft))
    print('Saving: {:.2f} s'.format(st))

    return sim_results

### Scenario runs

In the baseline scenario, customer demands are drawn from a Uniform(0,8) distribution

In [121]:
# Baseline simulation: demand uniformly distributed in [0,8]
baseline_sim = simulate(scenario = 'baseline', problem_sizes = [5], capacity = 20, route_size = 5, overlap_size = 5, cust_sims = 5, dem_sims = 500)
baseline_sim.head()

--- SIMULATION PARAMETERS ---
Scenario Name: baseline
Problem sizes: [5]
Vehicle capacity: 20
Primary route size: 5
Overlap size: 5
Customer instances: 5
Demand instances: 500

Starting problems of size 5
Creating customer instances...
Finding best tour across demand sets...
Starting customer instance 0...
Complete. Time elapsed: 4.24 s
Starting customer instance 1...
Complete. Time elapsed: 8.65 s
Starting customer instance 2...
Complete. Time elapsed: 13.69 s
Starting customer instance 3...
Complete. Time elapsed: 19.50 s
Starting customer instance 4...
Complete. Time elapsed: 25.98 s
Simulation complete.

--- RUNTIME BREAKDOWN ---
Setup: 0.60 s
Dedicated: 6.18 s
Overlapped: 6.35 s
Overlapped (w/ algorithm) 6.31 s
Full Flex.: 6.18 s
Saving: 0.36 s


,Scenario,Number of Customers,Routing Strategy,Metric,Value
0,baseline,5,dedicated,trip count,1.000000
1,baseline,5,dedicated,radial cost,73.874431
2,baseline,5,dedicated,circular cost,83.941829
3,baseline,5,dedicated,total cost,157.816260
4,baseline,5,overlapped (w/o alg),trip count,1.000000


### Simulation summary

In [66]:
# Combine all simulation results into single dataframe
combined = pd.concat([baseline_sim])

# Calculate averages over instances
means = combined.groupby(['Scenario', 'Number of Customers', 'Routing Strategy', 'Metric'])['Value'].mean()

### Save results

In [223]:
#date = time.strftime("%Y-%m-%d")
#outfile = 'output/results_{}.xlsx'.format(date)
timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")
outfile = 'output/results_{}.xlsx'.format(timestamp)
with pd.ExcelWriter(outfile) as writer:
    baseline_sim.to_excel(writer, sheet_name = 'baseline')
    means.to_excel(writer, sheet_name = 'summary_mean')